# Train model

The purpose of this notebook is:

- Load all training data (images and labels).
- Limit number of train images to `task.nr_train_images` (value provided as run parameter).
- Train a support vector machine model using sklearn.
- Persist the trained model using the ONNX format.

### Determine run parameters

In [1]:
# ----------------- Parameters for interactive development --------------
P = {
    "pipeline.data_lake_root": "/pipeline-outputs/data-lake",
    "task.nr_train_images": 600,
}

In [2]:
# - During automated runs parameters will be injected in the below cell -

In [3]:
# Parameters
P = {
    "pipeline.data_lake_root": "/pipeline-outputs/data-lake",
    "pipeline.run_environment": "ci",
    "pipeline.pipeline_run_id": "a6c4c4ae-4fdd-40fb-8a52-6e3e3984e925",
    "pipeline.github.repository": "pynb-dag-runner/mnist-digits-demo-pipeline",
    "pipeline.github.workflow": "Run automated tests, pipeline and deploy results to static reporting site",
    "pipeline.github.runner_name": "GitHub Actions 2",
    "pipeline.github.run_id": "2668450777",
    "pipeline.github.actor": "matiasdahl",
    "pipeline.github.job": "run-tests-pipeline-and-persist-pipeline-outputs",
    "pipeline.github.base_ref": "",
    "pipeline.github.head_ref": "",
    "pipeline.github.sha": "147a1be9916173fd4d2cd011d8d51e3411215850",
    "pipeline.github.ref": "refs/heads/development",
    "pipeline.github.ref_type": "branch",
    "pipeline.github.ref_name": "development",
    "pipeline.github.event_name": "schedule",
    "task.nr_train_images": 800,
    "task.notebook": "notebooks/train-model.py",
    "task.max_nr_retries": "1",
    "run.retry_nr": "0",
    "task.timeout_s": "None",
    "task.num_cpus": 1,
    "_opentelemetry_traceparent": "00-8ed032a1273b4d8d9ed85f02e5fe0471-673e622bbeda83d2-01",
}


In [4]:
# -----------------------------------------------------------------------

---

### Notebook code

In [5]:
from pynb_dag_runner.tasks.task_opentelemetry_logging import PydarLogger

logger = PydarLogger(P)

2022-07-14 17:37:32,984	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 172.17.0.2:6379


## Load and limit train data

In [6]:
def load_and_limit_train_data(P):
    from common.io import datalake_root, read_numpy
    from sklearn.model_selection import train_test_split

    X_train_all = read_numpy(datalake_root(P) / "train-data" / "digits.numpy")
    y_train_all = read_numpy(datalake_root(P) / "train-data" / "labels.numpy")

    assert isinstance(P["task.nr_train_images"], int)

    # Note: train_test_split will fail if split is 0 or 100%.
    assert 0 < P["task.nr_train_images"] < len(y_train_all)

    X_train, _, y_train, _ = train_test_split(
        X_train_all,
        y_train_all,
        train_size=P["task.nr_train_images"],
        test_size=None,
        stratify=y_train_all,
        shuffle=True,
        random_state=123,
    )

    assert X_train.shape == (len(y_train), 8 * 8)
    return X_train, y_train


X_train, y_train = load_and_limit_train_data(P)

## Train support vector classifier model

Below we assume that the hyperparameter $C$ is known.

However, this should ideally be found by a hyperparameter search. That could be
done in parallel on the Ray cluster, but this needs some more work. Ie., to use
multiple cores in the notebook, those cores should be reserved when starting the
notebook task (TODO).

- https://docs.ray.io/en/latest/tune/key-concepts.html

Note: cv-scores would need to be computed here, since they depend on the train data.
After this notebook only the onnx-model is available.

In [7]:
from sklearn.svm import SVC

In [8]:
model = SVC(C=0.001, kernel="linear", probability=True)

model.fit(X_train, y_train)

SVC(C=0.001, kernel='linear', probability=True)

### Q: Can the labels returned by `predict(..)` be computed from probabilities returned by the `predict_prob`-method?

In [9]:
import numpy as np

y_train_labels = model.predict(X_train)
y_train_probabilities = model.predict_proba(X_train)
assert y_train_probabilities.shape == (len(y_train), 10)

y_train_max_prob_labels = np.argmax(y_train_probabilities, axis=1)
assert y_train_labels.shape == y_train_max_prob_labels.shape == y_train.shape

# If the predicted labels would coincide with the labels that have
# maximum probability, the below number would be zero
logger.log_int(
    "nr_max_prob_neq_label", int(sum(y_train_max_prob_labels != y_train_labels))
)

 - Logging nr_max_prob_neq_label (int) : 1


The explanation is (likely) explained in the SVC source, see
[here](https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/svm/_base.py).
Namely, the outputs from `predict(..)` and `predict_proba(..)` may not in some
cases be compatible since the latter is computed using cross-validation while
the former is not. Thus, the above number need not be zero.

## Persist model

In [10]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

#
from common.io import datalake_root, write_onnx

In [11]:
# convert sklearn model into onnx and persist to data lake

model_onnx = convert_sklearn(
    model, initial_types=[("float_input_8x8_image", FloatTensorType([None, 8 * 8]))]
)
write_onnx(
    datalake_root(P)
    / "models"
    / f"nr_train_images={P['task.nr_train_images']}"
    / "model.onnx",
    model_onnx,
)